In [2]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import glob
%matplotlib inline

def decay(x,a,b,c):
    #c = tau    
    return a + b*np.exp(-x/c)

data_path = '20211104-0002/'
data_files = glob.glob(data_path + '*.csv')
ring_downs = []
raw_ring_downs = []

for i in data_files:
    data = pd.read_csv(i, skiprows=1) 
    start_full = data.loc[(data['(V)'] == max(data['(V)']))].last_valid_index()
    end_full = data.loc[(data['(us)'] >= 250)].first_valid_index()
    
    data2 = data[start_full:end_full]
    
    #need to do 0.9 and 0.1 times the voltage and find the times for those
    
    start = data2.loc[(data2['(V)'] <= max(data2['(V)'])*0.9)].first_valid_index()
    end = data2.loc[(data2['(V)'] <= max(data2['(V)'])*0.1)].first_valid_index()

    time = data['(us)']
    time = time[start_full:end_full]
    voltage = data['(V)']
    voltage = voltage[start_full:end_full]
    
    popt, pcov = curve_fit(decay, time, voltage)
    
    residuals = voltage - decay(time, *popt)
    res_ss = np.sum(residuals**2)
    ss_tot = np.sum((voltage - np.mean(voltage))**2)
    r_squared = 1 - (res_ss / ss_tot)
    
    raw_time = data2['(us)']
    raw_time = raw_time[start:end]
    raw_ringdown = max(raw_time)-min(raw_time)
    
    limit = 0.998
    if r_squared >= limit:
        ring_downs.append(popt[2]*2.198)
        raw_ring_downs.append(raw_ringdown)
    else:
        print('Rejecting poor fit.')
        

print('Minimum R-Squared: ', limit)    
print('Successful ringdown fits: ', len(ring_downs),)
print('Average calculated ringdown time: ', np.mean(ring_downs), ' (us)')
print('Average raw ringdown time: ', np.mean(raw_ring_downs), ' (us)')
print('Calculated ringdown Standard Deviation: ', np.std(ring_downs), ' (us)')


Rejecting poor fit.
Rejecting poor fit.
Minimum R-Squared:  0.998
Successful ringdown fits:  18
Average calculated ringdown time:  52.694713734596036  (us)
Average raw ringdown time:  51.031109667777784  (us)
Calculated ringdown Standard Deviation:  1.628987187833052  (us)
